## 0 - Introduction

This tutorial will gide through the map creation step,
where we will create a map (a configuration file),
that will tell `bidsme` how to bidsify any particular
data file.

The map creation is the most complicated part of bidsification pipeline.
Hopefully it must be done only once, and then created map file can be used
with some adaptations to several similar datasets. 

But first, we need load the paths to example dataset
defined in [dataset installation](./00-tutoral-paths.ipynb):

In [ ]:
%run "./Configuration/00-tutorial-paths.py"

If you see any error, please verify the path definitions in `../01-installation/00-paths.ipynb`.

We will also need a helper functions, defined in [Tools folder](../Tools/tools.py),
we just need to execute the file:

In [ ]:
%run "../Tools/tools.py"

Finally we will initialize `bidsme` and get the `logger` object
which will control the logging of all bidsme functions:

In [ ]:
import bidsme
logger = bidsme.init()

If the bidsme is installed in current environment, you should see
the version of `bidsme`.

## 1 - Project of bidsificated names

Due to complexity of map creation, it is importtant to know well
the dataset you try to bidsify, and have an draft for bidsified
names for the data.

Technically it is not required, but will allow to avoid a lot
of confusion later.

During the [preparation tutorial](01-basic-preparation.ipynb) `bidsme`
already organized the source data, and split it into sections ans series.

### First session

Taken the first subject `sub-001`, the first section contains the following
series:

In [ ]:
!ls $PREPARED_PATH/sub-001/ses-s01512/MRI

The `001-localizer` images are, as the name indicates, localizer technical images
and will be ignored.

The `002-cmrr_mbep2d_bold_mb2_task_nfat` and `008-cmrr_mbep2d_bold_mb2_rest` are
the functional MRI for the task (named nBack) and rest state, respectively.
Following
[BIDS for MRI](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#task-including-resting-state-imaging-data),
we will give the name similar to `func/sub-001_ses-s01512_task-nBack_dir-AP_bold.nii`.
For the rest state, we will just replace `task-nBack` by `task-rest`.

The `002-cmrr_mbep2d_bold_mb2_invertpe` and `006-cmrr_mbep2d_bold_mb2_invertpe` are
the fieldmaps taken for each of functional acquisition.
They corresponds to [peopolar fmap images](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#case-4-multiple-phase-encoded-directions-pepolar), so we will give them names similar to
`fmap/sub-001_ses-s01512_acq-nBack_dir-PA_epi.nii`.
Again replacing `acq-nBack` to `acq-rest` for the fieldmap for the resting state.

The `010-gre_field_mapping` and `011-gre_field_mapping` are the two magnitudes and
the phase difference images. Following the [BIDS](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#case-1-phase-difference-map-and-at-least-one-magnitude-image),
we will give them names `fmap/sub-001_ses-s01512_magnitude1.nii` and `fmap/sub-001_ses-s01512_phasediff`, respectively.

The `012-t1_mpr_sag_p2_iso` and `013-t2_spc_da-fl_sag_p2_iso`
are two anatomical T1-waited and Flair images.
Following the [BIDS](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#anatomy-imaging-data),
they will be named `anat/sub-001_ses-s01512_T1w.nii` and `anat/sub-001_ses-s01512_FLAIR.nii`

### Second session

Second session `ses-s01513` have a similar list of series:

In [ ]:
!ls $PREPARED_PATH/sub-001/ses-s01513/MRI

Ther localizer, functional MRI and fieldmaps are same as for
the first session, and will recieve the same names.

The anatomical images are replaced by a diffusion image and
two associated field maps.

The diffusion image `013-cmrr_mbep2d_diff_NODDI` will become
`dwi/sub-001_ses-s01513_dir-AP_dwi`, as [BIDS](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#diffusion-imaging-data) suggests.

The fieldmap `014-cmrr_mbep2d_diff_NODDI_noise` is a normal diffusion
image, but taken without an RF pulse, we will give it an `acq-RFoff`:
`fmap/sub-001_ses-s01513_acq-RFoff_dir-AP_epi.nii`.

The fieldmap `012-cmrr_mbep2d_diff_NODDI_invertpe` is similar to
the ones taken for the functional MRI, so we will give it similar names,
except for `acq-` entity, we will give `acq-RFoff`:
`fmap/sub-001_ses-s01513_acq-RFon_dir-PA_epi`

### Third session

The third session `ses-s01530` is the quantitative MRI (qMRI), inparticular
it follows multy-parametric mapping (MPM) protocols:

In [ ]:
!ls $PREPARED_PATH/sub-001/ses-s01530/MRI

Outside the localizer and `gre_field_mapping`, already discussed,
it contains 3 anatomical images: PD weighted, T1 weighted, and MTweighted, 
and two B1 fieldmaps taken for each of anatomical ones.

We will follow
[the bidsifications of qMRI](https://bids-specification.readthedocs.io/en/stable/99-appendices/11-qmri.html):

The `004-al_mtflash3d_PDw` and `005-al_mtflash3d_PDw` are
the anatomical images
(suffix -- `MPM`)
taken using several echo times (`echo-1` ... `echo-6`),
and splitted into magnitude and phase components (`part-mag` and `part-phase`).
Additionaly, as for the PD-weighted images, the MT pulse wasn't used, we will
add the `mt-off` entity.
We will also add `flip-1` to the name, to mark that PDw images uses different
flip angle from T
So the final name will become:
`anat/sub-001_ses-s01530_acq-PDw_echo-1_mt-off_part-mag_MPM.nii`.
`anat/sub-001_ses-s01530_acq-PDw_echo-1_flip-1_mt-off_part-mag_MPM.nii`

The `002-al_mtflash3d_sensArray` and `003-al_mtflash3d_sensBody`
are [B1 fieldmaps](https://bids-specification.readthedocs.io/en/stable/99-appendices/11-qmri.html#rb1cor-specific-notes)
(suffix -- `RB1COR`),
taken for the PD-weighted images, using head and body coils
(`acq-headPDw` and `acq-bodyPDw`).
So their names will be simply: `fmap/sub-001_ses-s01530_acq-headPDw_RB1COR.nii`.

The similar names can be applied to T1w and MTw images and corresponding fieldmaps,
using corresponding `acq-` entities `acq-T1w` and `acq-MTw`.
For MTw images we alse need to use the `mt-on` entity.

Finally, `014-al_B1mapping` is the
[global B1 map](https://bids-specification.readthedocs.io/en/stable/99-appendices/11-qmri.html#tb1epi-specific-notes)
(suffix -- `TB1EPI`)
sets of images, taken with two echo times (`echo-1`, `echo-2`)
and several flip angles (`flip-01`, ... `flip-08`).
So the final name will become: `fmap/sub-001_ses-s01530_echo-1_flip-01_TB1EPI`

For convinience, we put the full list of series and the projected bidsified
names into [../Files/projected_names.md](./Files/projected_names.md).

## 2 - Mapping

Before bidsifying the prepared dataset, `bidsme` needs to know precisely how to bidsify each 
data file. 
Instead of hard-coding all possible file names and protocols, `bidsme` will look
into the metadata associated with each individual image file, and will use the special file
`bidsmap.yaml`, to try and determine the BIDS-conforming name from it.

This approach allows the retrieval of any relevant information necessary for bidsification, regardless of naming conventions, protocols and parameters;  it is user's responsability to create the appropriate mapping file `bidsmap.yaml`.

### 1 - `bidsmap.yaml` file structure

This mapping file is written in [yaml](https://yaml.org/spec/1.2.2/) language, which uses tabs and spaces to denote substructures, similar to python.

A typical section of the `bidsmap` file is similar to the following
(see example in `$RESOURCES_PATH/map/bidsmap_noPlugin.yaml`):
```yaml
MRI:  # Type of data, determines the type of recording, e.g. MRI, PET, EEG etc. 
  hmriNIFTI: # Format of data, provides information to bidsme on how to read the metadata 
    func: # Modality, corresponds to sub-folders in BIDS subject/session directory; this is only defined for some type of data, such as MRI
      - provenance: /home/beliy/Works/bidscoin/test/renamed/sub-003/ses-HCL/MRI/002-cmrr_mbep2d_bold_mb2_invertpe/f1904-0002-00001-000001-01.nii # Path to (prepared) file used to check this entry
        example: func/sub-003_ses-HCL_task-nBack_dir-PA_run-1_bold # Example of bidsified name, generated for this block
        checked: true # manual user's approval that block is correct
        suffix: bold # suffix to use in bidsified name
        attributes: # list of attributes/metadata used to identify data
          ProtocolName: cmrr_mbep2d_bold_mb2_invertpe
          ImageType: ORIGINAL\\PRIMARY\\M\\MB\\ND\\MOSAIC
          SeriesNumber: 2
        bids: !!omap # list of bids entities used to create bidsified name
          - dir: PA
          - task: nBack
          - acq: ~
          - ce: ~
          - rec: ~
          - run: <AcquisitionNumber>
          - echo: ~
        json: !!omap # list of metadata to put into bidsified json file
          - TaskName: <<bids:task>>
          - RepetitionTime: <scale-3:RepetitionTime>
          - NumberOfVolumesDiscardedByScanner: 0
          - NumberOfVolumesDiscardedByUser: 0
          - DelayTime: 0
```

### 2 - Creating new `bidsmap.yaml` file

The actual mapping file `bidsmap.yaml` is created and updated by running `map` command.

In [ ]:
help(bidsme.mapper)

The `map` command will go trough the prepared dataset, and for each
image, will try to find a corresponding section in the `bidsmap.yaml`.

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

>By default `bidsmap.yaml` is created in `code/bidsme` folder in bidsified datast.
But a custom path to map file can be provided using `bidsmapfile` option.

> **CLI:** `bidsme map <source> <destination> [-b <bidsmap file>]`

The exact location of the created `bidsmap.yaml` is reported in the **log** output:
```
bidsmap._bidsmap(270) - INFO Writing bidsmap to: /home/beliy/Work/bidsme/bidsme_example/example1/bids/code/bidsme/bidsmap.yaml
```

We need to open it in an editor of choice and fix the appearing warnings and errors,
as described in the following chapters.

> **Note:** The yaml language is very sensitive to the syntax, in particular to the
tabs and spaces, that are used to create the hierachical structure.
Using an editor that highlights that structure and/or warns about syntax errors
is highly recommended!

### 3 - Treating auto-generated sections (template sections)

Bidsme will run untill it finds a image file that it unable to bidsify and stop,
showing found warnings and errors.
User should correct the warnings and errors and run map again untill all image files can be
bidsified.

The list of common warnings and erros, together with examples and possible corrections are in separate files [Errors.md](../Info/Errors.md) and [Warnings.md](../Info/Warnings.md).

The first **warning** you will probably see is:

```
mapper(73) - WARNING MRI/001-localizer/0: No run found in bidsmap. Looking into template
```

As indicated in [Warnings.md](../Warnings.md), bidsme encounted a new file that
it couldn't identify and created corresponding section in `bidsmap.yaml`.

This new section can be easely identifieble by field `template: true`:
```yaml
    __ignore__:
      - provenance: /home/beliy/Work/bidsme/bidsme_example/example1/renamed/sub-001/ses-s01512/MRI/001-localizer/s1512-0001-00001-000001-01.nii
        example: __ignore__/sub-001_ses-s01512
        template: true  # indicates that it was generated from template
        checked: false
        suffix: ''
        attributes:
          ProtocolName: 'localizer'
        bids: !!omap []
        json: !!omap []
```

There two things to do:

 1. Remove line containing `template: true`. This will allow to keep track of newly generated sections.
 2. Vrify that the file (in this case `MRI/001-localizer/s1512-0001-00001-000001-01.nii`, as cited in `provenance` field) is correctly identified. Here, the file is used for localisation and don't have much value in final dataset. Such files can be placed into `__ignore__` to tell to bidme to ignore and exclude them from bidsification.

 After these two steps are done, save `bidsmap.yaml`, and run `map` again:

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

On the next file `MRI/002-cmrr_mbep2d_bold_mb2_invertpe/f1512-0002-00001-000001-01.nii`
should produce more warnings and few errors.
But first two actions will remain the same as for previous file:

    1. Remove `template:true`
    2. Verify that `002-cmrr_mbep2d_bold_mb2_invertpe` is correctly attributed to fieldmap (`fmap`).

### 4 - Setting up bidsified name

Next step is to verify the bidsified name given in `example` field:
```yaml
    example: fmap/sub-001_ses-s01512_acq-cmrrmbep2dboldmb2invertpe_dir-PA_epi
```

The name is constructed in the `bids` subsection, where an ordered dictionary (`!!omap`)
is used to define each of entities.
The key corresponds to  the name of an entity, and the value to the value of entity.
If value is null value, represented by `~` in yaml, than this entity will be ignored.
And because it's an ordered dictionary, the entities will appear in the same order as
they appear in `bids` dictionary. 
So,
```yaml
    bids: !!omap
      - acq: <SeriesDescription>
      - ce: ~
      - dir: PA
      - run: ~
```
will produce bidsified name `acq-cmrrmbep2dboldmb2invertpe_dir-PA`,
to which the modality, subject, session and suffix will be added automaticaly to produce the
line that you can find in example field:
`fmap/sub-001_ses-s01512_acq-cmrrmbep2dboldmb2invertpe_dir-PA_epi`.

If we look for `002-cmrr_mbep2d_bold_mb2_invertpe` in the projected names,
we expect to have the name `fmap/sub-001_ses-s01512_acq-nBack_dir-PA_epi`.
We need just to set in the `bids` section:
```yaml
        bids: !!omap
          - acq: nBack
          - ce: ~
          - dir: PA
          - run: ~
```

Rerunning the `map` command, should fix the `example` field:

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

#### Duplicated bidsified name error

In the output of command, you should still see, the **error**:
```
mapper(105) - ERROR MRI/002-cmrr_mbep2d_bold_mb2_invertpe/1: Bidsified name same as first file of recording: fmap/sub-001_ses-s01512_acq-nBack_dir-PA_epi
```
This error happens becose second image in acquisition will end up with the same bidsified 
name as first image, so we need to modify `bids` sub-section again to attribute different names.

The easiest choice is to use different run index for each image:
```yaml
        bids: !!omap
          - acq: nBack
          - ce: ~
          - dir: PA
          - run: <AcquisitionNumber>
```
Bidsme will automatically replace tag `<AcquisitionNumber>` with the value of the field
`AcquisitionNumber` from file header
(search for it in `MRI/002-cmrr_mbep2d_bold_mb2_invertpe/f1512-0002-00001-000001-01.json`).

This will generate bidsified names:
 - fmap/sub-001_ses-s01512_acq-nBack_dir-PA_run-1_epi
 - fmap/sub-001_ses-s01512_acq-nBack_dir-PA_run-2_epi
 - fmap/sub-001_ses-s01512_acq-nBack_dir-PA_run-3_epi
 - fmap/sub-001_ses-s01512_acq-nBack_dir-PA_run-4_epi
 
and so remove the risk of overwritting files.

#### Formatting fields

These idsified names will work well if number of differen volumes is less than 10.
At 10th volume, the alphabetically ordered list of files will look like that:
 - fmap/sub-001_ses-s01512_acq-cmrrmbep2dboldmb2invertpe_dir-PA_run-1_epi
 - fmap/sub-001_ses-s01512_acq-cmrrmbep2dboldmb2invertpe_dir-PA_run-10_epi
 - fmap/sub-001_ses-s01512_acq-cmrrmbep2dboldmb2invertpe_dir-PA_run-2_epi
 - fmap/sub-001_ses-s01512_acq-cmrrmbep2dboldmb2invertpe_dir-PA_run-3_epi
 - fmap/sub-001_ses-s01512_acq-cmrrmbep2dboldmb2invertpe_dir-PA_run-4_epi
 
Which is not exactly convinient.

Bidsme allow some limited transformation of data retrieved from header, these transformations
are called `actions` and are defined in function `action_value` in file
`INSTALLATION_PATH/bidsme/Modules/common.py`.
In particular the `format` action will apply a particular 
[sring formating](https://docs.python.org/3/library/string.html#format-specification-mini-language).
What we need is to transform `1` to `001`, so larger indexes will not break alphabetical order. Using python string formatting, it will be achieved by `"{:03d}"`:

In [ ]:
"{:03d}".format(1)

In action formatting it is transformed to `<format03d:AcquisitionNumber>`:
 - `format` name of action
 - `03d` parameter of action
 - `AcquisitionNumber` field in metadata on which action will be applied
 
> If you made an error in name of action or in parameter, you will recieve an error similar 
to: Invalid field prefix format03s: Unknown format code 's' for object of type 'int',
where the first part of message will indicate what prefix/action was incorrect, and second part state the error itself

After this modification, the files from `MRI/002-cmrr_mbep2d_bold_mb2_invertpe` the error
should disappear:

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

### 5 - Replacing placeholders

Next step is to fix `Placehoder found` **warning**:
```
Modules.base(901) - WARNING 002-cmrr_mbep2d_bold_mb2_invertpe/0: Placehoder found
```

As indicated in  [Warnings.md](../Warnings.md), this warning is raised by `<<placeholder>>`
tag, which, in this case is in `json` subsection, in fields `NumberOfVolumesDiscardedByUser`
and `IntendedFor`:
```yaml
        json: !!omap
          - EffectiveEchoSpacing: <EffectiveEchoSpacing>
          - TotalReadoutTime: <TotalReadoutTime>
          - NumberOfVolumesDiscardedByUser: <<placeholder>>
          - RepetitionTime: <RepetitionTime>
          - IntendedFor:
              - "<<session>>/func/<<subject>>_<<session>>_task-<<placeholder>>_dir-AP_bold.nii"
```
This tag is here to mark important fields that BIDS require to fill.

#### NumberOfVolumesDiscardedByUser

The [NumberOfVolumesDiscardedByUser](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#timing-parameters_1)
is just count of manually discarded volumes (a.k.a dummy volumes) from fMRI
diffusion acquisition.

There no discarded volume for this fieldmap, so we can replace this placeholder by 0.

#### IntendedFor

The [IntendedFor](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#using-intendedfor-metadata)
field must contain the list of images, which will or can use a given fieldmap image.
The paths to the images must be relative to the subject folder,
if there are sessions in dataset, it should start with session,
otherwise, it should start with datatype folder.

The fieldmap, that we are currently setting was taken for functional fmri
`004-cmrr_mbep2d_bold_mb2_task_nfat`, 
that we expect to be bidsified under name
`func/sub-001_ses-s01512_task-nBack_dir-AP_bold`.
BIDS requires that we put the path to that file(s) in the IntendedFor filed.

Obviosly the path will be different for each subject and session, so instead of
`sub-001` and `ses-s01512` we will use tags `<<subject>>` and `<<session>>`.
Bidsme will automatically replace these tags with subject and session entities to current
file subject and session entities.

So the `IntendedFor` field should look similar to:
```yaml
- IntendedFor:
              - "<<session>>/func/<<subject>>_<<session>>_task-nBack_dir-AP_bold"
```

Running `map` again should remove the `placeholder` warnings.

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

If the `002-cmrr_mbep2d_bold_mb2_invertpe` don't produce any errors/warnings
`bidsme` will go to the next series: `002-cmrr_mbep2d_bold_mb2_invertpe`.

Use the procedure described above to setup the bidsified name
(check the projected names for reference), and to fix errors and warnings.

Once this series is accepted by `bidsme`, pass to the next step.

### 6 - Duplicated examples

When `bidsme` reaches `006-cmrr_mbep2d_bold_mb2_invertpe`, you should see:

**Error:**
```
mapper(382) - ERROR Matches example of already processed run fmap/sub-001_ses-s01512_acq-nBack_dir-PA_run-001_epi
```

It means that images from two different series will be bidssified
with exactly same name, or, equivalently, the same section in
`bidsmap.yaml` matches two different series.

It is easy to find the section in question, by searching in `bidsmap.yaml`
the line given in the warning message:
`fmap/sub-001_ses-s01512_acq-nBack_dir-PA_run-001_epi`.

The second series, that matches that section, is given just above the warning:
`006-cmrr_mbep2d_bold_mb2_invertpe`.

These are two fieldmaps, taken for task (nBack) and resting state fMRI.

`bidsme` determines which section applies to which image, by matching
image metadata to `attributes` sub-section.

In this case, the attributes checks are performed only on the name of the protocol:
```yaml
        attributes:
          ProtocolName: "cmrr_mbep2d_bold_mb2_invertpe"
```

Both fieldmaps uses the same protocol, so they are matched by same section,
and will get the same bidsified name.

The solution to this problem, is to find another metadata, which could
differentiate between these two series.
Comparing the image header dumps for both images:
`$PREPARED_PATH/sub-001/ses-s01512/MRI/002-cmrr_mbep2d_bold_mb2_invertpe/f1512-0002-00001-000001-01.json`
and
`$PREPARED_PATH/sub-001/ses-s01512/MRI/006-cmrr_mbep2d_bold_mb2_invertpe/f1512-0002-00001-000001-01.json`
you could realize that almost all parameters and names are the same,
except `SeriesNumber`.

So we just need to add check for `SeriesNumber` to attributes of section
mentioned in error message:
```yaml
        attributes:
          ProtocolName: "cmrr_mbep2d_bold_mb2_invertpe"
          SeriesNumber: 2
```
and run `map` again:

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

Now the `006-cmrr_mbep2d_bold_mb2_invertpe` creates it's own section in bidsmap,
and generates the same warnings and errors as `002-cmrr_mbep2d_bold_mb2_invertpe`.
Thse errors and warnings can be easely fixed using the procedure, described above.

> The second fieldmap is taken for fMRI in resting state, 
and `acq` and `task` should be `rest` and not `nBack`.

#### Images checking several sections

After fixing the usual errors and warnings, a new warning should appear:

**Warning:**
```
bidsmap._bidsmap(201) - WARNING fmap/0: also checks run: fmap/1
```

It means that same image will match two or more different sections in bidsmap.
In our case, `002-cmrr_mbep2d_bold_mb2_invertpe` (`fmap/0`, first section of fmap)
will match section created for `006-cmrr_mbep2d_bold_mb2_invertpe`
(`fmap/1`, second section of fmap).

We just need to add SeriesNumber criterium to attributes for the second section:
```yaml
        attributes:
          ProtocolName: "cmrr_mbep2d_bold_mb2_invertpe"
          SeriesNumber: 6
```

> **Warning:** Index of acquisition (SeriesNumber) is not very reliable. If there is an additional acquisition insetred, all the indexes will become shifted, so use indexes only it's the
only way distinguish acquisitions. More reliable way can also be implemented using plugins,
and covered in the [advanced mapping tutorial](05-advanced-bidsification.ipynb).

When all errors of `006-cmrr_mbep2d_bold_mb2_invertpe` are fixed,
continue to run the `map` step.

> **Note:** If you feel confident, and you know that the section for task
and resting state fMRI will be almost identical, you can just duplicate
this section, and adjust the copy to much the resting state
(ProtocolName, and task name).
The section starts with line `-provenance`, and be carefull to conserve
all tabs and spaces, yaml is very sensitive to them.

Continue untill you arrive to `010-gre_field_mapping`.

### 7 - Using metadata for bidsified name generation

Arriving to `010-gre_field_mapping`, you should see a new warning:

**Warning:**
```
Modules.base(600) - WARNING 010-gre_field_mapping/0: Can't find 'ScanOptions' attribute from '<ScanOptions>'
```
This warning just signals that one of the json fields of `MRI/010-gre_field_mapping`
uses `<ScanOptions>`,
but this field wasn't defined in the header, this field will be replaced
by an empty string.
It will disappear the next time you run the `map` command, and it is safe
to ignore it.

Also, `010-gre_field_mapping` and the following `011-gre_field_mapping` are taken for
the both anatomical T1-weighted and Flair images,
so the `IntendedFor` must reflect it:
```yaml
        json: !!omap
          - IntendedFor:
              - "<<session>>/func/<<subject>>_<<session>>_T1w.nii"
              - "<<session>>/func/<<subject>>_<<session>>_FLAIR.nii"
```

#### Dynamic suffix and entities generation

In the section of `010-gre_field_mapping`, you can see that,
similarly to the `run` entity we use image metadata to generate
bidsified name:
```yaml
        suffix: magnitude<EchoNumbers>
```
wich will, automatically generate the suffix `_magnitude1` or `_magnitude1`
depending on the `EchoNumbers` value of field map.

`bidsme` interprets a word enclosed in single `<>` as reference
to the corresponding metadata entry.

For the data format, used in this example, the metadata is stored
in a json file, having the same name as the image
(you can quickly accede to it by taking the path, given in `provenance`
field, and replace extention `.nii` to `.json`.
The metadata itself is just a dump of DICOM header, stored in `acqpar`
dictionary.

For some data formats, where metadata is not stored in open form
(DICOM, EDF), `bidsme` will automatically crete a dump of the header
in `json` file `bidsme_header_<filename>.json`.

Opening the header file, you can explore the metadata.
For example, for the `EchoNumbers`, used to generate the suffix:
```json
      "ImagedNucleus": "1H",
      "EchoNumbers": 1,
      "MagneticFieldStrength": 3,
```

Some of the metadata values are hidden in the nested structures,
dictionaries and lists.
To acess them you need to provide full path to a given value,
using `/` as separator.

For example, if you want to retrieve the nominal B0 field strenght,
stored in field `flNominalB0`, in `MrPhoenixProtocol`, you must
provide the path:
```
<MrPhoenixProtocol/sProtConsistencyInfo/flNominalB0>
```

Continue to process series, untill you will arrive to `004-cmrr_mbep2d_bold_mb2_task_fat`

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

### 8 - Re-using already created sections

When you will arrive to the task fMRI of the second session (`ses-s01513`),
you can take a shortcut.
We know that the tasks from first and second session will be bidsified
with the same name, and will have same metadata.

So, instead of creating and filling the second session for
`004-cmrr_mbep2d_bold_mb2_task_fat`, we can just make first section
of `func` mutch both series.

#### Providing list of valies

To do so, in `attributes` subsection, instead of one value associated
to `ProtocolName`, we can provide a list:
```yaml
        attributes:
          ProtocolName:
            - cmrr_mbep2d_bold_mb2_task_nfat
            - cmrr_mbep2d_bold_mb2_task_fat
```

`bidsme` then, will check if an image have `ProtocolName` matching
any of given values. 

Alternatively, you can use [regular expression](https://docs.python.org/3/library/re.html),
to match both:
```yaml
        attributes:
          ProtocolName: cmrr_mbep2d_bold_mb2_task_n?fat
```

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

Now the first func section (`func/0`) matches both `004-cmrr_mbep2d_bold_mb2_task_nfat`
and `004-cmrr_mbep2d_bold_mb2_task_fat` series.
The third func section (`func/2`) is unused, which is reported in:
**Warning:**
```
bidsmap._bidsmap(326) - WARNING MRI/hmriNIFTI/func[2]: No matched recordings
```
If you search for this section, you will see that the `provenance` field is empty:
```yaml
      - provenance: ~
```
meaning that none of the images matched it.
So we can safely remove this section from `bidsmap.yaml`.

Continue to process series, untill you will arrive to `002-al_mtflash3d_sensArray`

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

### 9 - Models and conformity to BIDS

> **Warning:** Information in this chapter may become obsolete for future
versions of `bidsme`!

#### Check for conformity to BIDS

`bidsme` maintains in memory the list of entities and fields for sidecar json.
If you set-up a bidsified name, that deviates from BIDS nameing schema,
you will see a warning:

**Warning:**
```
Modules.base(1137) - WARNING 002-al_mtflash3d_sensArray/0: Naming schema not BIDS
```

You can see it, if you add, for example `part` entity to the bids sub-section
of `002-al_mtflash3d_sensArray`.

This warning will persist, untill user manually confirms that this naming
is exactly what he expects to have, by setting `checked` field to `true`.

The role of `checked` will be discussed later.

#### Using different naming schemas 

The acquisistion `002-al_mtflash3d_sensArray` opens the quantitative MRI part of dataset,
which should be bidsified in a more
[complex way](https://bids-specification.readthedocs.io/en/latest/99-appendices/11-qmri.html).
Despite imgaes should be put in usual BIDS modality folders `anat` and `fmap`,
the naming schema is different, and different information is required
in the sidecar json.

Due to this different schema, the exotic protocols should always produce a warning,
which will become annoyng very quickly.

So, `bidsme` implements so called models, a variants of the naming schemas.
For example:
```yaml
        model: "mpm_RB1COR"
        suffix: RB1COR                                       
        attributes:                                          
          ProtocolName: al_mtflash3d_sensArray
```
will tell `bidsme` to not apply the standard schema, used for fieldmaps,
but a special schema, called `mpm_RB1COR`, and thus avoiding
the unnessesary warnings.

The naming schema and sidecar json fields for a given modality (in this case MRI)
are defined in `$INSTALLATION_PATH/bidsme/Modules/MRI/_MRI.py`. 
The list of entities is stored in `modalities` dictionary. 
If an image belongs for example to `anat`, bidsme will load the list of entities from 
`modalities["anat"]`.

The optional `model` field will foce to use different list of entities from
`modalities` dictionary.
We will use the models extensively, while creating map for MPM part
of the examle dataset.

Set the bidsified name following the draft, fix the usual warnings and continue,
until you arrive to `005-al_mtflash3d_PDw`

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

### 10 - Protecting attributes values

The series `005-al_mtflash3d_PDw` and `006-al_mtflash3d_PDw` is actually 
the same sets of images, but splitted into their phase and magnitude parts.

To resolve the `Matches example of already processed run` error, we should
find a metadata wich disinguish the magnitude and phase parts,
similar as we did for fieldmaps above.

Conviniently there is such field in the metadata: `ImageType`:
```json
"ImageType": "ORIGINAL\\PRIMARY\\M\\ND ", # for magnitude
"ImageType": "ORIGINAL\\PRIMARY\\P\\ND ", # for phase
```

So we just need to add corresponding entry to `attributes`:
```yaml
       attributes:
          ProtocolName: al_mtflash3d_PDw
          ImageType: "ORIGINAL\\PRIMARY\\M\\ND"
```

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

Doing so, a new error will appear:

**Error:**
```
mapper(388) - ERROR Error processing folder /home/beliy/Work/bidsme/bidsme_example/example1/renamed/sub-001/ses-s01530/MRI/004-al_mtflash3d_PDw in file s1530-0004-00001-000176-01.nii: bad escape \P at position 8
```

This rather cryptic error indicates that `bidsme` can't read one of the attributes
values.
As mentioned above, `bidsme` uses regular exppressions when matching attributes,
and the `ImageType` is set to `"ORIGINAL\\PRIMARY\\M\\ND"`.
The python tries interpret the litteral string as regular expression, and
stops at unknown escape character.

To tell the `bidsme` that it should use the `ORIGINAL\\PRIMARY\\M\\ND`,
as it is.
To do so, we need to put it inside single quotes `'`:
```yaml
        attributes:
          ProtocolName: al_mtflash3d_PDw
          ImageType: 'ORIGINAL\\PRIMARY\\M\\ND'
```

Now, the `004-al_mtflash3d_PDw` should be processed without errors.

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

Continue to process the series untill you arrive to `014-al_B1mapping`.

Please note that you will encounter 3 groups of `al_mtflash3d_sensBody`
and `al_mtflash3d_sensArray`, and you will need to use `SeriesNumber`
to distinguish them.

### 11 - Filling more placeholders

Similarly to other fieldmaps, the `014-al_B1mapping` section contains
several placeholders:
    - `flip`: indexing the various flip angles used in acquisition
    - `FlipAngleSeries`: The full list of flip angles values
    - `MixingTime`: Te B1 mixing time used
    
The `flip` index is easely retrievable from metadata, it corresponds to
`AcquisitionNumber`:
```yaml
        bids: !!omap
          - echo: <EchoNumbers>
          - flip: <format02d:AcquisitionNumber>
          - acq: ~
```

It is possible to retrieve the `FlipAngleSeries` and `MixingTime`,
but it would require the usage of plugins -- and it will be covered
in [the corresponding tutorial](05-advanced-bidsification.ipynb).

For now, we will just fill the placeholders by values that we know:
```yaml
          - FlipAngleSeries: [115, 110, 105, 100, 95]
          - MixingTime: 0.0338
```

The last placeholder is in the `IntendedFor` field, ad we must
put the paths to all 6 anatomical series: MTw, PDw and T1w:
```yaml
          - IntendedFor:                             
              - "<<session>>/anat/<<subject>>_<<session>>_acq-PDw_echo-*_flip-1_mt-off_part-mag_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-PDw_echo-*_flip-1_mt-off_part-phase_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-T1w_echo-*_flip-2_mt-off_part-mag_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-T1w_echo-*_flip-2_mt-off_part-phase_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-MTw_echo-*_flip-1_mt-on_part-mag_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-MTw_echo-*_flip-1_mt-on_part-phase_MPM.nii"
```

After setting `014-al_B1mapping`, the rest of series should be processed without
warnings:

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

## 3 - Finalyzing map

When you processed all series, and none of them are raising errors and warings,
you still should see a warning:

**Warning:**
```
mapper(441) - WARNING Map contains 25 unchecked runs
```

This warning tells that there 25 sections in `bidsmap.yaml`
that didn't recieved final user approval.

The last step for creating maps is to manually check and
confirm the validity of each of sections.

First section in your bidsmap should correspond to `localizer`, in `__ignore__`.
We just need to confirm that we want to exclude from bidsification
all localizer images.
To do so, we need to change the field `checked` to `true`:
```yaml
    __ignore__:
      - provenance: /home/beliy/Work/bidsme/bidsme_example/example1/renamed/sub-001/ses-s01512/MRI/001-localizer/s1512-0001-00001-000001-01.nii
        example: __ignore__/sub-001_ses-s01512
        checked: true
```

When running the `map` step again, the number of unchecked sections
should be 24:

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

### 1 - Filling sidecar json fields

The following unchecked section should correspond to `002-cmrr_mbep2d_bold_mb2_invertpe`.
But first we need to control and fill json fields.

The json subsection represents the metafields that will populate bidsified
image
[sidecar json](https://bids-specification.readthedocs.io/en/stable/02-common-principles.html#imaging-files).
`bidsme` track the
[reqired, recommended and optional fields](https://bids-specification.readthedocs.io/en/stable/02-common-principles.html#compulsory-optional-and-additional-data-and-metadata), and automatically populates the json sub-section.

The fields are filled as follows:
 - *REQUIRED* -- filled by `<<placeholder>>`, these fields will raise warnings
 and, in theory, are already filled during steps above
 - *RECOMMENDED* -- filled by an empty string `''`
 - *OPTIONAL* -- filled by null value `~`

Each fields in subsection can be filled by a raw value:
```yaml
          - NumberOfVolumesDiscardedByUser: 0
```
or by extracted value from image header, putting the name in `<>`:
```yaml
          - RepetitionTime: <RepetitionTime>
```
or by internal bidsme parameter, putting the name in `<<>>`:
```yaml
          - OriginalFile: <<filename>>
```

#### Creating non-bids complient sidecar json entries

Sometimes you will need some metadata to process your images,
but this metadata is not part of BIDS.

To do so, just add a new field into `json` sub-section:
```yaml
        json: !!omap
          - OriginalName: <<filename>>
          - EffectiveEchoSpacing: <EffectiveEchoSpacing>
```

In this example it will add a new field `OriginalName`, which
will contain the path to original non-bidsified image.

#### Combyning several meatadata into one string

If the value of metadata is a string, you can concatenate several
strings into one string.
To do so, just combine the text and parameters, similar to what we did to
`IntendedFor`:
```yaml
          - IntendedFor:
              - "<<session>>/func/<<subject>>_<<session>>_task-nBack_dir-AP_bold.nii"
```

If metadata is not string, the concatenetion will still work, but result
will automatically converted to string, using the default python parsing,
and may lead to unpredicted results.
In this case, you can use `format` modificator, to convert a numerical
value in controlled way: `<format03d:AcquisitionNumber>` will result
into `001`.

#### Using elements of bidsified name

`bidsme` allows also to retrive parts of bidsified names.
We already seen that `<<subject>>` and `<<session>>` will
retrieve the subject and session id respectively.
`<<suffix>>` will retrieve the suffix, and `<<bids:entity>>`
will return the corresponding entity:

- `<<subject>>`: `sub-001`
- `<<session>>`: `ses-s01512`
- `<<suffix>>`: `epi`
- `<<bids:acq>>`: `nBack`
- `<<bids:run>>`: `001`

#### Using internal bidsme values

`bidsme` allows to retrieve some of his internal variables to be used
in the `json` sub-section.
We alread seen that `<<filename>>` will retrieve the path to original
file, and `<<placeholder>>` will raise a warning.
Other internal variables can be retrieved using:

- `<<serieNumber>>`: The numero of the current series, same as number 
    in prepared folder name
- `<<serie>>`: The name of the current series, same as prepared folder
    name
- `<<index>>`: The index (starting with 0) of current image in the
    series
- `<<nfiles>>`: Total number of images in the current series
- `<<filename>>`: Path to the current prepared image
- `<<modality>>`: The datatype (anat, func, etc.) of current file
- `<<nodule>>`: The modality (MRI, PET, etc.) of current file
- `<<placeholder>>`: raises a warning

#### Providing a list of values to the `json` sub-section

If you need to provide a list to a json fild, you can do it in a standard
yaml way, either placing values in `[]`:
```yaml
          - FlipAngleSeries: [115, 110, 105, 100, 95]
```
or by placing values on new line, and preceding them with `-`:
```yaml
          - IntendedFor:
              - "<<session>>/anat/<<subject>>_<<session>>_acq-PDw_echo-*_flip-1_mt-off_part-mag_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-PDw_echo-*_flip-1_mt-off_part-phase_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-T1w_echo-*_flip-2_mt-off_part-mag_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-T1w_echo-*_flip-2_mt-off_part-phase_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-MTw_echo-*_flip-1_mt-on_part-mag_MPM.nii"
              - "<<session>>/anat/<<subject>>_<<session>>_acq-MTw_echo-*_flip-1_mt-on_part-phase_MPM.nii"
```

#### Providing user-defined values

`bidsme` allows to fill the sidecar json with user-defined values,
for example, based on a combination of image metadata. 
However, it requires a plugins usage, and covered in a [separate
tutorial](05-advanced-bidsification.ipynb)

When you filled all required sidecar json fields, it is good to run
the `map` step again, to check if `bidsme` can correctly interpret
new values:

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

If you see a warning similar to:

**Warning:**
```
Modules.base(619) - WARNING 002-cmrr_mbep2d_bold_mb2_invertpe/0: Can't find 'RepetitionTime2' attribute from '<RepetitionTime2>'
```
then you shoud fix the error in the corresponding metadata field,
here just remove `2` from `<RepetitionTime2>`.

### 2 - Validating the section

When you are happy with the filled json metadata, you need to do the final
check:

1. Check that file in the `provenance` field is corresponds well to the
attributed data type (fmap, anat, etc.)
2. Check that the bidsified name, given in `example` field is correct for
the file in the `provenance` field
3. Check that all needed json fields are filled

If all good, remove the empty metadata (meatdata with empty values `''`),
so final sidecar json will not be polluted by empy strings.

> **Note:** If all fields in the json sub-section are to be removed,
you need to give an empty ordered dictionary as value to the subection:
>```yaml
>        json: !!omap []
>```
>Elsewere you will render `bidsmap.yaml` unreadable by `bidsme`

Finally you need to change the value of the field `checked` to `true`:
```yaml
      - provenance: /home/beliy/Work/bidsme/bidsme_example/example1/renamed/sub-001/ses-s01512/MRI/002-cmrr_mbep2d_bold_mb2_invertpe/f1512-0002-00001-000001-01.nii
        example: fmap/sub-001_ses-s01512_acq-nBack_dir-PA_run-001_epi
        checked: true
```

This will validate the section. 

Continue to perform final checks to all section.

When all sections are validated, run the `map` step again, to ensure
you don't have any warnings:

In [ ]:
bidsme.mapper(PREPARED_PATH, BIDSIFIED_PATH)
bidsme.tools.info.reporterrors(logger)
bidsme.tools.info.reseterrors(logger)

## 4 - Conclusion

Once the `map` step don't produce any warnings, we are ready to
finally bidsify the dataset.

The `bidsmap.yaml` still may have need some minor corrections,
after inspection of bidsified datas, but the bulk of the work
is one.

It will be a good idea to save `bidsmap.yaml` into a safe place.
For the reference, my [bidsmap.yaml](../Files/MRI_bidsmap_prepared.yaml)
at the current stage is saved in `Files` directory in the root of tutorial.

Now we are ready to do the [bidsification](03-basic-bidsification.ipynb)